In [ ]:
import gradio as gr
import tensorflow as tf
from tensorflow.keras.models import load_model
from PIL import Image
import numpy as np
import os

# --- Configuration ---
MODEL_PATH = 'cat_dog_cnn_91_accuracy.h5'
IMAGE_SIZE = (150, 150) # Must match your training size
CLASS_LABELS = ['Cat', 'Dog'] # Labels based on generator's class_indices (Cat=0, Dog=1)
# ---------------------

# 1. Load the Model
try:
    model = load_model(MODEL_PATH)
    print(f" Model '{MODEL_PATH}' loaded successfully.")
except Exception as e:
    print(f" ERROR: Could not load model '{MODEL_PATH}'. Please ensure the file is in the same directory.")
    print(f"Error details: {e}")
    # Exit or raise error if the model is crucial
    raise

# 2. Define the Prediction Function
def predict_image_gradio(input_img):
    """
    Takes a PIL Image object (provided by Gradio) and returns the prediction result.
    """
    if input_img is None:
        return "Please upload an image."

    # 1. Pre-process the image
    img = input_img.convert('RGB')
    img = img.resize(IMAGE_SIZE)
    img_array = np.asarray(img)
    
    # Add batch dimension and normalize (divide by 255.0)
    # Shape changes from (150, 150, 3) to (1, 150, 150, 3)
    img_batch = np.expand_dims(img_array, axis=0) / 255.0

    # 2. Make Prediction
    # The output is a single value between 0 and 1 (due to sigmoid)
    raw_prediction = model.predict(img_batch, verbose=0)[0][0]
    
    # 3. Interpret Result (Cat=0, Dog=1)
    if raw_prediction < 0.5:
        # Closer to 0 means Cat
        label_index = 0
        confidence_score = 1.0 - raw_prediction
    else:
        # Closer to 1 means Dog
        label_index = 1
        confidence_score = raw_prediction

    # Create a dictionary for Gradio's Label output format
    # The keys must be the class labels
    predictions = {
        CLASS_LABELS[0]: 1.0 - confidence_score,  # The probability of the opposite class
        CLASS_LABELS[1]: confidence_score
    }
    # Adjust the dictionary to ensure the predicted class has the high confidence
    final_output = {
        CLASS_LABELS[0]: 1.0 - confidence_score if label_index == 1 else confidence_score,
        CLASS_LABELS[1]: confidence_score if label_index == 1 else 1.0 - confidence_score
    }
    
    return final_output

# 3. Create the Gradio Interface
iface = gr.Interface(
    fn=predict_image_gradio,
    inputs=gr.Image(type="pil", label="Upload Cat or Dog Image"),
    outputs=gr.Label(num_top_classes=2, label="Prediction"),
    title="Convolutional Cat vs. Dog Classifier",
    description="Upload an image to see if the CNN model classifies it as a Cat or a Dog.",
    examples=[
        # Optional: Add paths to example images here for quick testing
        # e.g., ["path/to/example_cat.jpg"],
        # e.g., ["path/to/example_dog.jpg"]
    ],
    allow_flagging='never' # Disables the "Flag" button
)
iface.launch()


/Users/amitpatel/Developer/PythonFiles/Image-Classification-Cat-v-Dog/.venv/lib/python3.13/site-packages/gradio/interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


 Model 'cat_dog_cnn_91_accuracy.h5' loaded successfully.
* Running on local URL:  http://127.0.0.1:7866
* To create a public link, set `share=True` in `launch()`.


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/Users/amitpatel/Developer/PythonFiles/Image-Classification-Cat-v-Dog/.venv/lib/python3.13/site-packages/uvicorn/protocols/http/h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        self.scope, self.receive, self.send
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    )
    ^
  File "/Users/amitpatel/Developer/PythonFiles/Image-Classification-Cat-v-Dog/.venv/lib/python3.13/site-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/amitpatel/Developer/PythonFiles/Image-Classification-Cat-v-Dog/.venv/lib/python3.13/site-packages/fastapi/applications.py", line 1082, in __call__
    await super().__call__(scope, receive, send)
  File "/Users/amitpatel/Developer/PythonFiles/Image-C